# Transformation to C Code (Simple)

## Imports

In [1]:
# | code-fold: true
# | code-summary: "Load packages"
# | output: false

import os
import numpy as np

from library.fvm.solver import Settings
from library.model.models.shallow_water import ShallowWaterEquations

import library.model.initial_conditions as IC
import library.model.boundary_conditions as BC
from library.misc.misc import Zstruct
import library.transformation.to_c as trafo

2025-08-26 06:00:47.824 | WARNING  | library.fvm.solver:<module>:28 - No module named 'precice'


## Model definition

In [2]:
bcs = BC.BoundaryConditions(
    [
        BC.Wall(physical_tag="top"),
        BC.Wall(physical_tag="bottom"),
        BC.Wall(physical_tag="left"),
        BC.Wall(physical_tag="right"),
    ]
)

def custom_ic(x):
    Q = np.zeros(3, dtype=float)
    Q[0] = np.where(x[0] < 5., 0.005, 0.001)
    return Q

ic = IC.UserFunction(custom_ic)

model = ShallowWaterEquations(
    dimension=2,
    boundary_conditions=bcs,
    initial_conditions=ic,
)



## Code transformation

In [3]:

settings = Settings(name="ShallowWater", output=Zstruct(directory="outputs/trafo", filename="swe.h5"))
trafo.to_c(model, settings)

2025-08-26 06:00:48.330 | WARNING  | library.misc.misc:__init__:146 - No 'clean_directory' attribute found in output Zstruct. Default: False


## Check the output

In [4]:
main_dir = os.getenv("ZOOMY_DIR")
path = os.path.join(main_dir, os.path.join(settings.output.directory, '.c_interface/model.h'))
with open(path, "r") as f:
    print(f.read())



#pragma once
#include <AMReX_Array4.H>
#include <AMReX_Vector.H>



AMREX_GPU_HOST_DEVICE
AMREX_FORCE_INLINE
amrex::SmallMatrix<amrex::Real,3,1>
flux_x ( amrex::SmallMatrix<amrex::Real,3,1> const& Q,
    amrex::SmallMatrix<amrex::Real,2,1> const& Qaux) noexcept
    {
    amrex::Real t0 = 1.0/Q(0);
    res(0,0) = Q(1);
    res(1,0) = (1.0/2.0)*9.81*amrex::Math::pow(Q(0), 2) + amrex::Math::pow(Q(1), 2)*t0;
    res(2,0) = Q(1)*Q(2)*t0;
    return res;
    }
        


AMREX_GPU_HOST_DEVICE
AMREX_FORCE_INLINE
amrex::SmallMatrix<amrex::Real,3,1>
flux_y ( amrex::SmallMatrix<amrex::Real,3,1> const& Q,
    amrex::SmallMatrix<amrex::Real,2,1> const& Qaux) noexcept
    {
    amrex::Real t0 = 1.0/Q(0);
    res(0,0) = Q(2);
    res(1,0) = Q(1)*Q(2)*t0;
    res(2,0) = (1.0/2.0)*9.81*amrex::Math::pow(Q(0), 2) + amrex::Math::pow(Q(2), 2)*t0;
    return res;
    }
        


AMREX_GPU_HOST_DEVICE
AMREX_FORCE_INLINE
amrex::SmallMatrix<amrex::Real,3,1>
flux_jacobian_x ( amrex::SmallMatrix<amrex::Real,3